# Laboratorio: Modelos del lenguaje con RNNs

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Portada_Trafalgar_%281873%29.jpg/800px-Portada_Trafalgar_%281873%29.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano de Trafalgar, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro Trafalgar, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [2]:
%pip install keras
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import joblib

In [2]:
path = keras.utils.get_file(
fname="Trafalgar.txt",
  origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ"
)

Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [14]:
with open(path, 'r', encoding='utf-8') as file:
  text = file.read().lower()

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [15]:
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])

Longitud del texto: 300039
-i-

se me permitirá que antes de referir el gran suceso de que fui testigo,
diga algunas palabras sobre mi infancia, explicando por qué extraña
manera me llevaron los azares de la vida a presenciar la terrible
catástrofe de nuestra marina.

al hablar de mi nacimiento, no imitaré a la mayor parte de


## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*Se me permitirá que antes de referir el gr*" -> predicción: **a**
* "*e me permitirá que antes de referir el gra*" -> predicción: **n**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: e me permitirá que antes de referir el gra

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: a

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante (PERO USANDO PALABRAS NO CARACTERES).

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de PALABRAS con la siguiente PALABRA a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* PALABRAS (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de las palabras y mapas de palabras

Antes que nada, necesitamos saber qué palabras aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_words* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_words* - 1. Por ejemplo, {'se': 0, 'me': 1, ...}
3.   Diccionario reverso de índices a palabras: {0: 'se', 1: 'me', ...}


In [28]:
texto=""
for letra in text:
	if not letra in "?¿.,'¡!()-{};_|[]»«:\"":
		texto+=letra
text=texto
palabras = text.split()
palabras_unicas = set(palabras)
word_index = {}
k=0
for palabra in palabras_unicas:
	word_index[palabra] = k
	k+=1

print(palabras_unicas)
print(word_index)

{'semejante', 'verticales', 'zenit', 'intentaron', 'vaya', 'gesto', 'traerme', 'del', 'europeas', 'llorando', 'concluía', 'bailarín', 'genio', 'federicodecía', 'aficionado', 'siga', 'desalojando', 'cargadas', 'herían', 'belleza', 'representaba', 'emprendió', 'pesadumbres', 'contemplando', 'rodilla', 'convertía', 'dispuesto', 'hijo', 'teníamos', 'chingurria', 'refuerzos', 'dentro', 'embriagarse', 'polvos', 'limitan', 'ánima', 'tranquilo', 'grumetes', 'principios', 'aplausos', 'suavemente', 'soplaron', 'enerofebrero', 'avemaría', 'ah', 'nieto', 'tripulación', 'mezquinamente', 'admiradores', 'dirigíamos', 'aquellas', 'salté', 'diálogo', 'cuyo', 'déjalesme', 'gastón', 'convenientemente', 'preocupado', 'pasajero', 'argel', 'dices', 'saldría', 'conversaciones', 'tomándome', 'existía', 'inoportunas', 'componía', 'metralla', 'comencé', 'vigilar', 'acobardó', 'nací', 'rival', 'portalón', 'primeras', 'calma', 'escuchaba', 'color', 'lucido', 'asomaron', 'mostrándome', 'cuadernas', 'dilatada', 'ed

#### 2.2. Obtención de secuencias de entrada y palabra a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y las correspondientes palabras a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH palabras y la siguiente palabra a predecir. Una vez hecho, desplazarse una palabra a la derecha y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y las palabras a predecir en una variable ***next_words***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 2, tendríamos

* *sequences* = ["Don Quijote", "Quijote de"]
* *next_chars* = ['de', 'La']

In [29]:
SEQ_LENGTH = 30
# Espacio entre sentencias
step=1 # Para que la siguiente sentencia esté desplazada un caracter a la izquierda.
sequences = []
next_words = [] # siguiente caracter a predecir para la sentencia asociada.

for i in range(0,len(text)-SEQ_LENGTH, step):
  sequences.append(text[i:i+SEQ_LENGTH])
  next_words.append(text[i+SEQ_LENGTH])

Indicar el tamaño del training set que acabamos de generar.

In [30]:
print('número de datos de training:{}'.format(len(sequences)))

número de datos de training:289593


Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [31]:
MAX_SEQUENCES = 1000

perm = np.random.permutation(len(sequences))
sequences, next_chars = np.array(sequences), np.array(next_words)
sequences, next_chars = sequences[perm], next_chars[perm]
sequences, next_chars = list(sequences[:MAX_SEQUENCES]), list(next_chars[:MAX_SEQUENCES])

print(len(sequences))

1000


#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestras palabras. Por ejemplo, si sólo tuviéramos 4 palabras (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_words)* e **y** tendrá shape *(num_sequences, num_words)*.



In [42]:
NUM_CHARS = len(palabras)  # Tu número de caracteres distintos aquí
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, NUM_CHARS))
y = np.zeros((NUM_SEQUENCES, NUM_CHARS))

for i,sequence in enumerate(sequences): # i es el número de secuencia y sequence contiene los caracteres 
  for t,palabra in enumerate(sequence): # t recorre la longitud de la secuencia y palabra cada caracter de la secuencia 
    X[i,t,word_index[palabra]]=1
  y[i,word_index[next_words[i]]]=1

KeyError: ' '

In [38]:
for j in range(SEQ_LENGTH):
  for k in range(NUM_CHARS):
    if (X[0,j,k]==1):
      print(word_index[k],end='')

erqué la mecha y el cañón disp

## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [39]:
vocab_size=len(palabras)

In [40]:
model=Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH,vocab_size)))
model.add(Dense(vocab_size,activation='softmax'))
print(model.summary())

C:\Users\Javi\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │        92,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 51)             │         6,579 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,739 (385.70 KB)

 Trainable params: 98,739 (385.70 KB)

 Non-trainable params: 0 (0.00 B)

None


In [42]:
optimizer='adam'
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [43]:
history=model.fit(X,y, validation_split=0.05, batch_size=128, epochs=20,shuffle=True).history

Epoch 1/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 35s 22ms/step - accuracy: 0.2338 - loss: 2.6957 - val_accuracy: 0.3270 - val_loss: 2.2095
Epoch 2/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.3449 - loss: 2.1436 - val_accuracy: 0.3749 - val_loss: 2.0541
Epoch 3/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 35s 23ms/step - accuracy: 0.3751 - loss: 2.0273 - val_accuracy: 0.3851 - val_loss: 1.9877
Epoch 4/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 35s 24ms/step - accuracy: 0.3960 - loss: 1.9574 - val_accuracy: 0.3996 - val_loss: 1.9392
Epoch 5/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 32s 22ms/step - accuracy: 0.4117 - loss: 1.9054 - val_accuracy: 0.4164 - val_loss: 1.8909
Epoch 6/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 33s 22ms/step - accuracy: 0.4272 - loss: 1.8608 - val_accuracy: 0.4255 - val_loss: 1.8545
Epoch 7/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 33s 22ms/step - accuracy: 0.4430 - loss: 1.8104 - val_accuracy: 0.4344 - val_loss: 1.8232
Epoch 8/20
1485/1485 ━━━━━━━━━━━━━━━━━━━━ 34s 23ms/step - accuracy: 0.4550 -

In [27]:
joblib.dump(history,'modelo_entrenado.pkl')
model2=joblib.load('modelo_entrenado.pkl')

Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo podemos ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtenemos un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que nuestros resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [29]:
def sample(probs, temperature=1.0):
    """Nos da el índice del elemento a elegir según la distribución
    de probabilidad dada por probs.
    
    Args:
      probs es la salida dada por una capa softmax:
        probs = model.predict(x_to_predict)[0]
      
      temperature es un parámetro que nos permite obtener mayor
        "diversidad" a la hora de obtener resultados. 
        
        temperature = 1 nos da la distribución normal de softmax
        0 < temperature < 1 hace que el sampling sea más conservador,
          de modo que sampleamos cosas de las que estamos más seguros
        temperature > 1 hace que los samplings sean más atrevidos,
          eligiendo en más ocasiones clases con baja probabilidad.
          Con esto, tenemos mayor diversidad pero se cometen más
          errores.
    """
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')
    
    # Hacemos logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature
    
    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    
    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)

Utilizando la función anterior y el modelo entrenado, vamos a añadir un callback a nuestro modelo para que, según vaya entrenando, veamos los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tenéis disponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Completa la función *generate_text* de modo que utilicemos el modelo y la función sample para generar texto.

NOTA: Cuando hagas model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [30]:
#CÓDIGO Ejemplo: Predice el siguiente caracter de una secuencia aleatoria
# Primero, seleccionamos una secuencia al azar para empezar a predecir
# a partir de ella
start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
seed_text = text[start_pos:start_pos + SEQ_LENGTH]
print("Seed: {}".format(seed_text))
print()
#----------------------------------
# Aquí representomos la secuencia seed_text como codificación one-hot
X_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS))
# Construimos X_pred a partir de seed_text
for t,char in enumerate(seed_text): # t recorre la longitud de la secuencia y char cada caracter de la secuencia 
  X_pred[0,t,char_indices[char]]=1
#-------------------------------------
#PREDICCIÓN
prediccion=model.predict(X_pred, batch_size=32, verbose=0)
print(indices_char[np.argmax(prediccion)],end='') # Muestra el caracter más probable para la sequencia 100
##################################################333

Seed:  un buque, no
es tan fácil de 

l

In [31]:
print(seed_text)
seed_text=seed_text[1:SEQ_LENGTH-1]
print(seed_text)
seed_text+=indices_char[np.argmax(prediccion)]
print(seed_text)

 un buque, no
es tan fácil de 
un buque, no
es tan fácil de
un buque, no
es tan fácil del


In [ ]:
def genera_texto(texto, model, length):
  X_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS)) # Declaramos X_pred
  for k in range(length): # Genera el número de caracteres fijados en el parámetro length
    #Inicializamos X_pred a cero
    for a in range(SEQ_LENGTH):
      for b in range(NUM_CHARS):
         X_pred[0,a,b]=0
    # Aquí representomos la secuencia seed_text como codificación one-hot
    # Construimos X_pred a partir de seed_text 
    for t,char in enumerate(texto): # t recorre la longitud de la secuencia y char cada caracter de la secuencia 
      X_pred[0,t,char_indices[char]]=1
    #-------------------------------------
    #PREDICCIÓN
    prediccion=model.predict(X_pred, batch_size=32, verbose=0)
    print(indices_char[np.argmax(prediccion)],end='') # Muestra el caracter más probable para la sequencia dada
    #Actualizamos la cadena de entrada
    texto=texto[1:SEQ_LENGTH]
    texto+=indices_char[np.argmax(prediccion)]


In [ ]:
start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
seed_text = text[start_pos:start_pos + SEQ_LENGTH]
print('Seed:',seed_text)
print('Texto generado:')
genera_texto(seed_text, model2, 120)

In [ ]:
TEMPERATURES_TO_TRY = [0.2, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length, temperature=1):
    """Genera una secuencia de texto a partir de seed_text utilizando model.
    
    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """
    
    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text
    
    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.
 
    texto=seed_text
    X_pred = np.zeros((1, SEQ_LENGTH, NUM_CHARS)) # Declaramos X_pred
    for k in range(length): # Genera el número de caracteres fijados en el parámetro length
      #Inicializamos X_pred a cero
      for a in range(SEQ_LENGTH):
        for b in range(NUM_CHARS):
          X_pred[0,a,b]=0
      # Aquí representomos la secuencia seed_text como codificación one-hot
      # Construimos X_pred a partir de seed_text 
      for t,char in enumerate(texto): # t recorre la longitud de la secuencia y char cada caracter de la secuencia 
        X_pred[0,t,char_indices[char]]=1
      #-------------------------------------
      #PREDICCIÓN
      prediccion=model.predict(X_pred, batch_size=32, verbose=0)
      # print(indices_char[np.argmax(prediccion)],end='') # Muestra el caracter más probable para la sequencia dada
      #Actualizamos la cadena de entrada
      texto=texto[1:SEQ_LENGTH]
      #texto+=indices_char[np.argmax(prediccion)]
      texto+=indices_char[sample(prediccion[0],temperature)]
      # generated+=indices_char[np.argmax(prediccion)]  
      generated+=indices_char[sample(prediccion[0],temperature)]
    ######################################################################
    ### FIN DE TU CÓDIGO
    return generated

In [ ]:
# Primero, seleccionamos una secuencia al azar para empezar a predecir
# a partir de ella
start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
seed_text = text[start_pos:start_pos + SEQ_LENGTH]
print('Seed:',seed_text)
print('Texto generado:')
generate_text(seed_text, model2, GENERATED_TEXT_LENGTH,temperature=0.1)

In [ ]:



def on_epoch_end(epoch, logs):
  print("\n\n\n")
  
  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))
    
    generated_text = generate_text(seed_text, model2, 
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))
    print()


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)  

In [ ]:
history=model2.fit(X,y, validation_split=0.05, batch_size=128, epochs=2,shuffle=True,verbose=0,callbacks=[generation_callback]).history

Entrena ahora tu modelo. No te olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [ ]:
## TU CÓDIGO AQUÍ

## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2 usando El Quijote:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.


